<a href="https://colab.research.google.com/github/MichalRyszardWojcik/hello-world/blob/master/trax_2020_08_20_second_day.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Based on https://github.com/jalammar/jalammar.github.io/blob/master/notebooks/Trax_TransformerLM_Intro.ipynb

The example from there returns a flipped array.

My example is basically the same but it returns an array filled with the maximum or minimum value.

[a,b,c,d] --> [m,m,m,m] where m = max(a,b,c,d) or m = min(a,b,c,d)

In [2]:
import os
import numpy as np
! pip install -q -U trax
import trax

     |████████████████████████████████| 368kB 2.8MB/s 
     |████████████████████████████████| 2.6MB 9.1MB/s 
     |████████████████████████████████| 163kB 28.6MB/s 
     |████████████████████████████████| 1.5MB 27.0MB/s 
     |████████████████████████████████| 778kB 41.7MB/s 
     |████████████████████████████████| 1.1MB 49.3MB/s 
     |████████████████████████████████| 3.5MB 43.1MB/s 
     |████████████████████████████████| 51kB 6.4MB/s 
     |████████████████████████████████| 307kB 48.3MB/s 
     |████████████████████████████████| 184kB 38.7MB/s 
     |████████████████████████████████| 655kB 45.5MB/s 
     |████████████████████████████████| 358kB 45.6MB/s 
     |████████████████████████████████| 81kB 8.7MB/s 
     |████████████████████████████████| 5.3MB 48.1MB/s 
     |████████████████████████████████| 368kB 49.9MB/s 
     |████████████████████████████████| 983kB 49.5MB/s 
     |████████████████████████████████| 3.0MB 46.7MB/s 
     |████████████████████████████████| 890kB 46.6MB/s

In [3]:
# Create a Transformer model.
def tiny_transformer_lm(mode='train'):
  return trax.models.TransformerLM(  
          d_model=32, d_ff=128, n_layers=2, 
          vocab_size=32, mode=mode)

In [4]:
def max_ints_task(batch_size, length=4):
  while True:
    random_ints = m = np.random.randint(1, 31, (batch_size,length))
    source = random_ints

    maxline = np.amax(source,1).reshape(1,batch_size)
    max = np.copy(maxline)
    for i in range(length-1):
      max = np.concatenate((max,maxline),0)
    target = max.transpose()

    zero = np.zeros([batch_size, 1], np.int32)
    x = np.concatenate([zero, source, zero, target], axis=1)

    loss_weights = np.concatenate([np.zeros((batch_size, length+2)),
                                    np.ones((batch_size, length))], axis=1)
    yield (x, x, loss_weights)  # Here inputs and targets are the same.

In [55]:
def maxmin_ints_task(batch_size, length=4):
  while True:
    random_ints = m = np.random.randint(1, 31, (batch_size,length))
    source = random_ints

    coin = np.random.randint(0,2)
    if (coin == 0):
      maxline = np.amax(source,1).reshape(1,batch_size)
    else:
      maxline = np.amin(source,1).reshape(1,batch_size)
    max = np.copy(maxline)
    for i in range(length-1):
      max = np.concatenate((max,maxline),0)
    target = max.transpose()

    zero = np.zeros([batch_size, 1], np.int32)
    x = np.concatenate([zero, source, zero, target], axis=1)

    loss_weights = np.concatenate([np.zeros((batch_size, length+2)),
                                    np.ones((batch_size, length))], axis=1)
    yield (x, x, loss_weights)  # Here inputs and targets are the same.

In [67]:
a = maxmin_ints_task(8)
sequence_batch, _ , masks = next(a)
sequence_batch


array([[ 0, 30, 13, 28, 28,  0, 13, 13, 13, 13],
       [ 0,  7, 22,  4, 26,  0,  4,  4,  4,  4],
       [ 0, 23, 17, 18,  9,  0,  9,  9,  9,  9],
       [ 0, 24,  9, 14, 26,  0,  9,  9,  9,  9],
       [ 0, 11,  5,  5, 17,  0,  5,  5,  5,  5],
       [ 0, 29, 30, 14,  6,  0,  6,  6,  6,  6],
       [ 0,  5,  4, 22, 17,  0,  4,  4,  4,  4],
       [ 0, 26,  8, 18,  5,  0,  5,  5,  5,  5]])

In [69]:
ints_inputs = trax.data.inputs.Inputs(lambda _: maxmin_ints_task(16))

In [70]:
output_dir = os.path.expanduser('~/train_dir/')
!rm -f ~/train_dir/model.pkl.gz  # Remove old model.

# Train tiny model with Trainer.
trainer = trax.supervised.Trainer(
    model=tiny_transformer_lm,
    loss_fn=trax.layers.CrossEntropyLoss(),
    optimizer=trax.optimizers.Adafactor,  # Change optimizer params here.
    lr_schedule=trax.lr.constant(0.001),  # Change lr schedule here.
    inputs=ints_inputs,
    output_dir=output_dir)

# Train for 3 epochs each consisting of 500 train batches, eval on 2 batches.
n_epochs  = 3
train_steps = 800
eval_steps = 2
for _ in range(n_epochs):
  trainer.train_epoch(train_steps, eval_steps)


Step    800: Ran 800 train steps in 55.80 secs
Step    800: Evaluation
Step    800: train                   accuracy |  0.87500000
Step    800: train                       loss |  0.53858292
Step    800: train         neg_log_perplexity | -0.53858292
Step    800: train          sequence_accuracy |  0.50000000
Step    800: train weights_per_batch_per_core |  64.00000000
Step    800: eval                    accuracy |  0.82031250
Step    800: eval                        loss |  0.77849197
Step    800: eval          neg_log_perplexity | -0.77849197
Step    800: eval           sequence_accuracy |  0.28125000
Step    800: eval  weights_per_batch_per_core |  64.00000000
Step    800: Finished evaluation

Step   1600: Ran 800 train steps in 7.22 secs
Step   1600: Evaluation
Step   1600: train                   accuracy |  0.84375000
Step   1600: train                       loss |  0.47890729
Step   1600: train         neg_log_perplexity | -0.47890729
Step   1600: train          sequence_accur

In [ ]:
input = np.array([[0, 4, 6, 29, 10, 0]])

# Initialize model for inference.
predict_model = tiny_transformer_lm(mode='predict')
predict_signature = trax.shapes.ShapeDtype((1,1), dtype=np.int32)
predict_model.init_from_file(os.path.join(output_dir, "model.pkl.gz"),
                             weights_only=True, input_signature=predict_signature)

# Run the model
outputt = trax.supervised.decoding.autoregressive_sample(
    predict_model, input, temperature=0.0, max_length=4)

# Print the contents of output
outputt

array([[29, 29, 29, 29]])

In [8]:
def output(input):
  predict_model = tiny_transformer_lm(mode='predict')
  predict_signature = trax.shapes.ShapeDtype((1,1), dtype=np.int32)
  predict_model.init_from_file(os.path.join(output_dir, "model.pkl.gz"),weights_only=True, input_signature=predict_signature)
  return trax.supervised.decoding.autoregressive_sample(predict_model, input, temperature=0.0, max_length=4)

In [74]:
#input = np.array([[0, 4, 6, 29, 10, 0]])
#output(input)

def randominput():
  x = np.random.randint(1, 31, (1,6))
  x[0,0] = 0; x[0,5] = 0
  return x

def randomdemo():
  a = randominput()
  b = output(a)
  return [a,b]

for _ in range(7): print(randomdemo())

[array([[ 0, 30, 19, 25, 14,  0]]), array([[30, 30, 30, 30]])]
[array([[ 0, 30,  5, 10,  3,  0]]), array([[3, 3, 3, 3]])]
[array([[ 0,  3, 16, 11,  1,  0]]), array([[1]])]
[array([[ 0,  2, 23, 27,  4,  0]]), array([[2, 2, 2, 2]])]
[array([[ 0, 12, 11, 20,  8,  0]]), array([[8, 8, 8, 8]])]
[array([[ 0, 16, 12, 17,  5,  0]]), array([[5, 5, 5, 5]])]
[array([[ 0,  2, 28,  3,  9,  0]]), array([[3, 3, 3, 3]])]


As expected it sometimes finds max and sometimes min, but why the output length is not fixed?
<pre><code>
[array([[ 0, 22,  6, 12, 13,  0]]), array([[22, 22, 22, 22]])]
[array([[ 0,  1, 23, 13, 16,  0]]), array([[1]])]
[array([[ 0,  8,  1, 18,  4,  0]]), array([[1]])]
[array([[ 0, 24, 30, 19, 30,  0]]), array([[30, 30, 30, 30]])]
[array([[ 0,  7,  2,  3, 19,  0]]), array([[2, 2, 2, 2]])]
[array([[ 0, 10,  9, 29,  4,  0]]), array([[29, 29, 29, 29]])]
[array([[ 0,  1,  3, 20, 12,  0]]), array([[1]])]
<br>
[array([[ 0, 10,  2,  3,  6,  0]]), array([[2, 2, 2, 2]])]
[array([[ 0, 15,  9, 27, 27,  0]]), array([[27, 27, 27, 27]])]
[array([[ 0, 20, 12,  1, 24,  0]]), array([[1]])]
[array([[ 0, 25, 19, 17,  1,  0]]), array([[1]])]
[array([[ 0,  5, 27,  6,  7,  0]]), array([[27, 27, 27, 27]])]
[array([[ 0, 15, 12,  1, 21,  0]]), array([[1]])]
[array([[ 0,  4, 12, 23, 23,  0]]), array([[23, 23, 23, 23]])]
</code></pre>